In [1]:
# Text text processing library
import torchtext
from torchtext.vocab import Vectors
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import itertools as it
from models import *
from helpers import *
debug = False

In [2]:
# Our input $x$
TEXT = torchtext.data.Field()

# Data distributed with the assignment
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(
    path=".", 
    train="train.txt", validation="valid.txt", test="valid.txt", text_field=TEXT)

TEXT.build_vocab(train)
if debug:
    TEXT.build_vocab(train, max_size=1000)

train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=10, device=-1, bptt_len=32, repeat=False)

# Build the vocabulary with word embeddings
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
TEXT.vocab.load_vectors(vectors=Vectors('wiki.simple.vec', url=url))

In [11]:
train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=5, device=-1, bptt_len=32, repeat=False, shuffle=True)
def inspect_batch(batch):
    for i in range(batch.text.data.size(1)):
        print(' '.join([TEXT.vocab.itos[i] for i in batch.text[:,i].data]))
        # print(' '.join([TEXT.vocab.itos[i] for i in batch.text[:,5].data]))
it = iter(train_iter)
for i in range(2):
    batch = next(it)
    print(batch.text.data.size(0))
    inspect_batch(batch)
    print("EOB")
# print(batch.text[:,3])


32
aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter <eos> pierre <unk> N years old will join
in part because of buy programs generated by stock-index arbitrage a form of program trading involving futures contracts <eos> but interest <unk> as the day wore on and investors looked ahead to
recorders and personal computers and then sell them at a huge <unk> <eos> the going rate for a small personal computer that costs about $ N in the west is anywhere from
comes across as a <unk> executive mr. phillips has a <unk> <unk> <eos> during time off mr. roman tends to his garden mr. phillips <unk> to a <unk> for among other things
with notes they 're <unk> distinct <eos> dean witter reynolds inc. lost its second recent arbitration case involving a former <unk> executive <eos> a new york stock exchange arbitration panel ordered dean
EOB
32
the board as a nonexecutive directo

Perplexity goals:
count: 120-200
feedforward: 100-150
recurrent: below 100 (between 80-100)

In [245]:
train_iter, _, _  = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=1, device=-1, bptt_len=10000, repeat=False)
tgram = Trigram(TEXT)
tgram.train_counts(train_iter)
tgram.set_alpha(0.25, 0.25)

Iteration 0


In [269]:
_, val_iter, _  = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=1, device=-1, bptt_len=10000, repeat=False)
for (a1, a2) in [(0.3, 0.5)]: #it.product(np.arange(0.1, 1, 0.1), repeat=2):
    if a1 + a2 >= 1:
        continue
    tgram.set_alpha(a1, a2)
    le = LangEvaluator(tgram, TEXT, evalmetric='perplexity')
    print(a1, a2, le.evaluate(val_iter))

Iteration 0
0.3 0.5 191.339042201


In [332]:
params_train = list(filter(lambda p : p.requires_grad, model_nnlm.parameters()))
print([p.size() for p in params_train])

[torch.Size([10001, 300]), torch.Size([60, 1, 6, 300]), torch.Size([60]), torch.Size([10001, 1, 6, 300]), torch.Size([10001]), torch.Size([10001, 60]), torch.Size([10001])]


In [330]:
train_iter, _, _  = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=10, device=-1, bptt_len=32, repeat=False)
model_nnlm = NNLM(TEXT)
trainer = LangTrainer(TEXT, model_nnlm)
trainer.train(train_iter)

Iteration 0, loss: 9.210577, norm: 0.000000
Iteration 10, loss: 9.138780, norm: 0.000000
Iteration 20, loss: 9.048756, norm: 0.000000
Iteration 30, loss: 8.949280, norm: 0.000000
Iteration 40, loss: 8.841352, norm: 0.000000
Iteration 50, loss: 8.572618, norm: 0.000000
Iteration 60, loss: 8.503356, norm: 0.000000
Iteration 70, loss: 8.215203, norm: 0.000000
Iteration 80, loss: 7.986099, norm: 0.000000
Iteration 90, loss: 7.916728, norm: 0.000000


In [3]:
train_iter, _, _  = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=4, device=-1, bptt_len=36, repeat=False)
model_lstm = LSTMLM2(TEXT)
trainer = LangTrainer(TEXT, model_lstm, use_hidden=True, lrn_rate=1.0)
trainer.train(train_iter, num_iter=3)

CUDA is unavailable...
FEATURE
aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter <eos> pierre <unk> N years old will join the board as
soar though monetary aggregates also grew so rapidly <unk> issued <unk> warnings of inflation <eos> but this <unk> the u.s. manufacturing sector laying the seeds of <unk> <eos> mr. lawson though not allowed to join
government can ensure the same flow of resources and reduce the current deficit <eos> predictably guarantees outstanding have risen by $ N billion since N while direct loans outstanding have fallen by $ N billion
it was during those hearings that the nation first learned the <unk> and <unk> of gen. noriega 's <unk> and of his <unk> relationships with a variety of u.s. government agencies <eos> those hearings also
LABEL
banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx n

torch.Size([2, 4, 650]) torch.Size([2, 4, 650]) HIDDEN SIZE
FEATURE
years ago researchers reported <eos> the asbestos fiber <unk> is unusually <unk> once it enters the <unk> with even brief exposures to it causing symptoms that show up decades later researchers said <eos> <unk> inc. the
<unk> of <unk> <eos> we had been soliciting opinions on it long before mr. lawson 's resignation and offer some of the collection for the benefit of his successor and one-time deputy john major <eos> to
average about $ N billion annually according to the congressional budget office <eos> a budget that reflected the real costs of lending would eliminate incentives to convert spending or lending programs to guarantees and would let
noriega on the shelf until nicaragua was settled <eos> as the report issued by the subcommittee concluded our government did nothing regarding gen. noriega 's drug business and substantial criminal involvement because the first priority was
LABEL
ago researchers reported <eos> 

torch.Size([2, 4, 650]) torch.Size([2, 4, 650]) HIDDEN SIZE
FEATURE
heard of asbestos having any questionable properties <eos> there is no asbestos in our products now <eos> neither <unk> nor the researchers who studied the workers were aware of any research on smokers of the kent
in britain gives its economy a structural bias toward inflation <eos> inflation is easier to spark and harder to control <eos> we should also concede that in the british experience the <unk> cause <unk> some of
as N the <unk> bank held in its portfolio at face value loans made to cuba in the 1950s <eos> more seriously <unk> has carried several billion dollars of defaulted loans at face value <eos> until
to develop any plan military or economic for supporting the panamanian people in their attempts to restore democracy <eos> sen. john kerry d. mass <eos> for vietnamese these are tricky often <unk> times <eos> after years
LABEL
of asbestos having any questionable properties <eos> there is no asbestos in our pro

torch.Size([2, 4, 650]) torch.Size([2, 4, 650]) HIDDEN SIZE
FEATURE
different type of <unk> in N <eos> from N to N N billion kent cigarettes with the filters were sold the company said <eos> among N men who worked closely with the substance N have died
chancellor for being too easy because of a desire to <unk> sterling but also led the chorus saying that mr. <unk> was too tight when he let the dollar rise <eos> somewhere in between there must
to be a way to <unk> spending figures hide fraudulent activity or provide large subsidies <eos> the reforms described above would provide a more limited but <unk> safer and ultimately more useful role for government as
used to be an <unk> in <unk> a private entrepreneur <eos> his business success made him an official target in <unk> days <eos> mr. chan now N years old invented a <unk> <unk> he and his
LABEL
type of <unk> in N <eos> from N to N N billion kent cigarettes with the filters were sold the company said <eos> among N men who worked closel

torch.Size([2, 4, 650]) torch.Size([2, 4, 650]) HIDDEN SIZE
FEATURE
workers at the west <unk> mass. paper factory appears to be the highest for any asbestos workers studied in western industrialized countries he said <eos> the plant which is owned by <unk> & <unk> co. was
one <unk> by the domestic political <unk> of west germany <eos> perhaps the shock would have been less if they 'd fixed to another <unk> <unk> <unk> economy <eos> alan reynolds of <unk> adds his <unk>
a line-item veto as a test to determine whether the courts will rule that he has such authority <eos> mr. bush has long <unk> for passage of a bill or a constitutional amendment that would <unk>
reforms gathered pace and for <unk> they did <eos> after years of <unk> mr. chan produced a <unk> bicycle tire that <unk> its <unk> rival <eos> by N he was selling thousands of tires <eos> newspapers
LABEL
at the west <unk> mass. paper factory appears to be the highest for any asbestos workers studied in western industrialized co

torch.Size([2, 4, 650]) torch.Size([2, 4, 650]) HIDDEN SIZE
FEATURE
such as <unk> that are classified as <unk> according to <unk> t. <unk> a professor of <unk> at the university of vermont college of medicine <eos> more common <unk> fibers are <unk> and are more easily
inflation than to cause recession <eos> we would add that in political terms mrs. thatcher 's problem was failing to decide between the chancellor and her adviser <eos> in the end neither policy was followed and
exercise a line-item veto <eos> such an <unk> most likely would bring about a court challenge from congress that would clarify whether a president already has such power <eos> white house spokesman marlin fitzwater confirming comments
lost and i won he says <eos> he was further questioned to determine if he was a real working man or an <unk> <eos> says mr. chan when i showed it was from my own brain
LABEL
as <unk> that are classified as <unk> according to <unk> t. <unk> a professor of <unk> at the university of v

torch.Size([2, 4, 650]) torch.Size([2, 4, 650]) HIDDEN SIZE
FEATURE
workers dumped large <unk> <unk> of the imported material into a huge <unk> poured in cotton and <unk> fibers and <unk> mixed the dry fibers in a process used to make filters <eos> workers described clouds
sparked by remarks in <unk> from sir alan 's <unk> in the american economist a <unk> academic journal <eos> but it was the underlying situation that became <unk> <eos> what mr. major and mrs. thatcher will
equipment <eos> mr. <unk> succeeds william r. <unk> jr. who will remain with the company until the end of the year to support the transition and to complete important projects <eos> the bush administration said it
one of mr. chan 's sons <eos> after six months on the run mr. chan learned the order for his arrest had been canceled <eos> he <unk> his family in january N and began the long
LABEL
dumped large <unk> <unk> of the imported material into a huge <unk> poured in cotton and <unk> fibers and <unk> mixed the dr

torch.Size([2, 4, 650]) torch.Size([2, 4, 650]) HIDDEN SIZE
FEATURE
mutual funds continued to slide amid signs that portfolio managers expect further declines in interest rates <eos> the average seven-day compound yield of the N taxable funds tracked by <unk> 's money fund report eased a
neither aggregates nor interest rates can do <eos> this was the version of <unk> economics that in the late 1970s and early '80s worked in america and world-wide to solve a far more serious <unk> than
by allowing some countries to convert <unk> barriers into tariffs that together with existing tariffs then would be phased out over N years <eos> trade representative carla hills who along with agriculture secretary <unk> <unk> unveiled
authorities allowed mr. chan to resume work <eos> by late last year he was invited back as the tire king to display his products at a national exhibition <eos> national leaders stopped by his stand to
LABEL
funds continued to slide amid signs that portfolio managers expect

KeyboardInterrupt: 

In [8]:
le = LangEvaluator(model_lstm, TEXT, use_hidden=True)
print(le.evaluate(train_iter))
print(le.evaluate(test_iter))

Using CUDA for evaluation...
Validation time: 20.806038 seconds
2411.5569330249964
Validation time: 4.735303 seconds
2414.4938698270835


In [10]:
parameters = model_lstm.parameters()
parameters = list(filter(lambda p: p.grad is not None, parameters))
total_norm = max(p.grad.data.abs().max() for p in parameters)
print(total_norm)
print(trainer.training_norms)

0.07247233390808105
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


## IGNORE STUFF BELOW HERE

In [128]:
    # NOT USED!
    # Here arr is a 1-D numpy array; this returns 
    # groups of n consecutive words (with overlapping)
    def get_ngrams(self, arr, n=3):
        len_ngrams = arr.shape[0] - n + 1
        ngram_inds = np.tile(np.reshape(np.arange(len_ngrams), [len_ngrams, 1]), [1, n]) + \
                    np.tile(np.reshape(np.arange(n), [1, n]), [len_ngrams, 1])
        return np.take(arr, ngram_inds)

In [288]:
a = [1,2,3]
b = [2 * x + i for i,x in enumerate(a)]
print(b)

[2, 5, 8]


In [285]:
print(TEXT.vocab.vectors.size())

torch.Size([10001, 300])


In [5]:
pad_token = TEXT.vocab.stoi['<pad>']
it = iter(train_iter)
cnt = 0
for batch in it:
    cnt += 1
    if (len(np.where(batch.text.data.numpy() == pad_token)[0])):
        print(batch.text.data, cnt)

In [6]:
def f(x):
    print(x)

a = [1]
f(*a)

1
